In [1]:
## Instalaciones

%pip install torch
%pip install open3d
%pip install tabulate


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
## Dependencias

import torch
import os
import open3d as o3d
import numpy as np
import torch.optim as optim
from torch.utils.data import DataLoader
from model import PointnetClassifier, PointNetLoss
from modelnet10 import ModelNetClass, ModelNet, DatasetType
from utils.csv import save_loss_dict
from IPython.display import clear_output
from tabulate import tabulate


DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using {DEVICE}.")

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


/home/antonioskorin/.local/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Using cpu.


In [2]:
# parámetros de ejecución
checkpoint_freq = 25        # cada cuantos epoch guardar el modelo
ROOT_DIR = os.getcwd()
CHECKPOINT_DIR = os.path.join(ROOT_DIR, "checkpoint")

# parámetros del dataset
classes = [label for label in ModelNetClass]
batch_size = 32
dim = 3
num_points = 1024
num_classes = len(classes)

# hiperparámetros
num_global_feats = 1024     # número de features globales calculadas
epochs = 500
learning_rate = 0.001
reg_weight = 0.001
gamma = 2                   # Recomendado por el paper de focal loss


train_data = ModelNet(classes, DatasetType.TRAIN)
validation_data = ModelNet(classes, DatasetType.VALIDATION)
test_data = ModelNet(classes, DatasetType.TEST)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(validation_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

classifier = PointnetClassifier(dim, num_points, num_global_feats, num_classes).to(DEVICE)

optimizer = optim.Adam(classifier.parameters(), lr=learning_rate)

if DEVICE == "cuda":
    scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.0001, max_lr=0.01, step_size_up=2000, cycle_momentum=False)
else:
    scheduler = None
    
# TODO: Más adelante usar alpha para clases imbalanceadas
criterion = PointNetLoss(alpha=None, gamma=gamma, reg_weight=reg_weight, size_average=True).to(DEVICE)

In [3]:
def print_epoch_table_jupyter(loss_dict):
    clear_output(wait=True)  # Clear previous cell output
    table_data = []
    for epoch in range(len(loss_dict["train"]["loss"])):
        table_data.append([
            f"Epoch {epoch + 1}",
            round(loss_dict["train"]["loss"][epoch], 4),
            round(loss_dict["train"]["acc"][epoch], 4),
            round(loss_dict["valid"]["loss"][epoch], 4),
            round(loss_dict["valid"]["acc"][epoch], 4),
        ])
    headers = ["Epoch", "Train Loss", "Train Acc", "Val Loss", "Val Acc"]
    print(tabulate(table_data, headers=headers, tablefmt="fancy_grid"))


In [4]:
loss_dict = {
    "train": {
        "loss": list(),
        "acc": list()
    },
    "valid": {
        "loss": list(),
        "acc": list()
    }
}

best_model_loss = float('inf')
best_model_acc = 0.0
best_model_epoch = -1

for epoch in range(1, epochs + 1):
    print(f"Procesando Epoca {epoch}")
    '''
    Entrenamiento
    '''
    classifier = classifier.train()

    train_batch_losses = list()
    train_batch_accs = list()
    
    for pcds, labels in train_loader:
        pcds = pcds.to(DEVICE)
        labels = labels.squeeze().to(DEVICE)

        # Gradientes en cero
        optimizer.zero_grad()
        
        # Hacer predicciones, calcular pérdida
        out, _, A = classifier(pcds)
        loss = criterion(out, labels, A, is_train=True)

        # Calcular gradiente y optimizar
        loss.backward()
        optimizer.step()
        
        # Uso de scheduler para el LR, en caso de estar habilitado
        if DEVICE == "cuda":
            scheduler.step()

        with torch.no_grad():
            # Calculamos las elecciones
            pred_choice = torch.softmax(out, dim=1).argmax(dim=1)
            
            # Elecciones correctas
            correct = pred_choice.eq(labels.data).cpu().sum()
            accuracy = correct.item() / float(pcds.size(0))
                
    
        # Registramos métricas para el batch
        train_batch_losses.append(loss.item())
        train_batch_accs.append(accuracy)
        

    '''
    Validación
    '''
    with torch.no_grad():
        classifier = classifier.eval()

        valid_batch_losses = list()
        valid_batch_accs = list()
        
        for pcds, labels in validation_loader:
            pcds = pcds.to(DEVICE)
            labels = labels.squeeze().to(DEVICE)
            
            # Hacer predicciones
            out, _, A = classifier(pcds)
            loss = criterion(out, labels, A, is_train=False)

            # Calculamos las elecciones
            pred_choice = torch.softmax(out, dim=1).argmax(dim=1)
            
            # Elecciones correctas
            correct = pred_choice.eq(labels.data).cpu().sum()
            accuracy = correct.item() / float(pcds.size(0))

            # Registramos métricas para el batch
            valid_batch_losses.append(loss.item())
            valid_batch_accs.append(accuracy)

    # Loss y pérdida training
    epoch_train_loss = np.mean(train_batch_losses)
    epoch_train_acc = np.mean(train_batch_accs)
    loss_dict["train"]["loss"].append(epoch_train_loss)
    loss_dict["train"]["acc"].append(epoch_train_acc)
    # Loss y pérdida validation
    epoch_valid_loss = np.mean(valid_batch_losses)
    epoch_valid_acc = np.mean(valid_batch_accs)
    loss_dict["valid"]["loss"].append(epoch_valid_loss)
    loss_dict["valid"]["acc"].append(epoch_valid_acc)

    print_epoch_table_jupyter(loss_dict)
    
    if epoch % checkpoint_freq == 0:
        path = os.path.join(CHECKPOINT_DIR, f"model_epoch_{str(epoch).zfill(4)}.pth")
        torch.save(classifier.state_dict(), path)
    if epoch_valid_loss < best_model_loss:
        best_model_loss = epoch_valid_loss
        best_model_acc = epoch_valid_acc
        best_model_epoch = epoch
        path = os.path.join(CHECKPOINT_DIR, f"best_model_{str(epoch).zfill(4)}.pth")
        torch.save(classifier.state_dict(), path)
    

save_loss_dict(loss_dict)
print(f"Best performance achieved in epoch {best_model_epoch}: loss = {best_model_loss:.4f}, accuracy = {best_model_acc:.4f}")

╒═══════════╤══════════════╤═════════════╤════════════╤═══════════╕
│ Epoch     │   Train Loss │   Train Acc │   Val Loss │   Val Acc │
╞═══════════╪══════════════╪═════════════╪════════════╪═══════════╡
│ Epoch 1   │       0.8395 │      0.6393 │     0.3582 │    0.7723 │
├───────────┼──────────────┼─────────────┼────────────┼───────────┤
│ Epoch 2   │       0.5674 │      0.7116 │     0.4565 │    0.7239 │
├───────────┼──────────────┼─────────────┼────────────┼───────────┤
│ Epoch 3   │       0.5779 │      0.7109 │     0.3786 │    0.7314 │
├───────────┼──────────────┼─────────────┼────────────┼───────────┤
│ Epoch 4   │       0.4386 │      0.7648 │     0.348  │    0.7391 │
├───────────┼──────────────┼─────────────┼────────────┼───────────┤
│ Epoch 5   │       0.4537 │      0.7426 │     0.2518 │    0.8052 │
├───────────┼──────────────┼─────────────┼────────────┼───────────┤
│ Epoch 6   │       0.4073 │      0.7721 │     0.2626 │    0.7971 │
├───────────┼──────────────┼─────────────┼──────

# Dataset Testeo

In [ ]:
with torch.no_grad():
    classifier = classifier.eval()
    correct = 0
    
    for pcds, labels in test_loader:
        pcds = pcds.to(DEVICE)
        labels = labels.squeeze().to(DEVICE)
        
        # Hacer predicciones
        out, _, _ = classifier(pcds)
    
        # Calculamos las elecciones
        pred_choice = torch.softmax(out, dim=1).argmax(dim=1)
        
        # Elecciones correctas, acumuladas
        correct += pred_choice.eq(labels.data).cpu().sum().item()

    test_acc = correct / float(len(test_data))
    print("Test Acc:\t", test_acc)

Test Acc:	 0.9162995594713657


In [8]:
def test_it(path):
    classifier = PointnetClassifier(dim, num_points, num_global_feats, num_classes).to(DEVICE)
    classifier.load_state_dict(torch.load(path))

    with torch.no_grad():
        classifier = classifier.eval()
        correct = 0
        
        for pcds, labels in test_loader:
            pcds = pcds.to(DEVICE)
            labels = labels.squeeze().to(DEVICE)
            
            # Hacer predicciones
            out, _, _ = classifier(pcds)
        
            # Calculamos las elecciones
            pred_choice = torch.softmax(out, dim=1).argmax(dim=1)
            
            # Elecciones correctas, acumuladas
            correct += pred_choice.eq(labels.data).cpu().sum().item()

        test_acc = correct / float(len(test_data))
        print(f"{path}\t\t", test_acc)

# Tests
for i in range(25, 500 + 1, 25):
    path = os.path.join(CHECKPOINT_DIR, f"model_epoch_{str(i).zfill(4)}.pth")
    test_it(path)
path = os.path.join(CHECKPOINT_DIR, "best_model.pth")
test_it(path)

/home/matias/modelNet10classifier/checkpoint/model_epoch_0025.pth		 0.8612334801762115
/home/matias/modelNet10classifier/checkpoint/model_epoch_0050.pth		 0.8392070484581498
/home/matias/modelNet10classifier/checkpoint/model_epoch_0075.pth		 0.8788546255506607
/home/matias/modelNet10classifier/checkpoint/model_epoch_0100.pth		 0.9063876651982379
/home/matias/modelNet10classifier/checkpoint/model_epoch_0125.pth		 0.9052863436123348
/home/matias/modelNet10classifier/checkpoint/model_epoch_0150.pth		 0.9008810572687225
/home/matias/modelNet10classifier/checkpoint/model_epoch_0175.pth		 0.8854625550660793
/home/matias/modelNet10classifier/checkpoint/model_epoch_0200.pth		 0.9030837004405287
/home/matias/modelNet10classifier/checkpoint/model_epoch_0225.pth		 0.9129955947136564
/home/matias/modelNet10classifier/checkpoint/model_epoch_0250.pth		 0.9129955947136564
/home/matias/modelNet10classifier/checkpoint/model_epoch_0275.pth		 0.9052863436123348
/home/matias/modelNet10classifier/checkpoin